In [ ]:
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()

In [22]:
def unified_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [165]:
X, y = unified_shuffle(digits['data'], digits['target'])
split = int(y.size * 0.8)
X_train, y_train, X_val, y_val = X[:split].T/255, y[:split], X[split:].T/255, y[split:]

In [190]:
def init_params(X,y,n_neurons):
    n_classes = np.unique(y).size
    W1 = np.random.randn(n_neurons, X.shape[0]) * 0.01
    b1 = np.random.randn(n_neurons,1)
    W2 = np.random.randn(n_classes,n_neurons) * 0.01
    b2 = np.random.randn(n_classes,1)
    return W1, b1, W2, b2

def ReLu(Z):
    return np.maximum(0,Z)

def soft_max(Z):
    exps = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # Normalize to prevent overflow
    return exps / np.sum(exps, axis=0, keepdims=True) 

def forward(W1,b1,W2,b2,X):
    Z1 = W1.dot(X) + b1
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = soft_max(Z2)
    return Z1, A1, Z2, A2

def deriv_ReLU(Z):
    return Z > 0

def one_hot(Y):
    one_hot_y = np.zeros((Y.size,np.unique(Y).size))
    ## loop trhough each entry and then set the corresponding value to 1
    one_hot_y[np.arange(Y.size),Y] = 1
    return one_hot_y.T

def backward(Z1,A1,A2,W2,X,y):
    m = y.size
    one_hot_y = one_hot(y)
    dZ2 = A2 - one_hot_y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2,1,keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1,1,keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [191]:
def get_prediction(A2):
    return np.argmax(A2,0)

def get_acc(predictions,y_true):
    return np.sum(predictions == y_true)/y_true.size

def grad_descent(X,y,iterations,alpha,n_neurons):
    W1, b1, W2, b2 = init_params(X,y,n_neurons)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward(W1,b1,W2,b2,X)
        dW1, db1, dW2, db2 = backward(Z1,A1,A2,W2,X,y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iteration", i)
            print("Current Accuracy:", get_acc(get_prediction(A2),y))
    return W1, b1, W2, b2

In [194]:
W1, b1, W2, b2 = grad_descent(X_train.T, y_train, 1000, 0.1, 128)

Iteration 0
Current Accuracy: 0.09035
Iteration 50
Current Accuracy: 0.6057666666666667
Iteration 100
Current Accuracy: 0.7905166666666666
Iteration 150
Current Accuracy: 0.8454
Iteration 200
Current Accuracy: 0.8701166666666666
Iteration 250
Current Accuracy: 0.8819333333333333
Iteration 300
Current Accuracy: 0.8886833333333334
Iteration 350
Current Accuracy: 0.8944833333333333
Iteration 400
Current Accuracy: 0.8980666666666667
Iteration 450
Current Accuracy: 0.9015
Iteration 500
Current Accuracy: 0.9042166666666667
Iteration 550
Current Accuracy: 0.9064833333333333
Iteration 600
Current Accuracy: 0.9092333333333333
Iteration 650
Current Accuracy: 0.9114666666666666
Iteration 700
Current Accuracy: 0.91335
Iteration 750
Current Accuracy: 0.9148833333333334
Iteration 800
Current Accuracy: 0.9165166666666666
Iteration 850
Current Accuracy: 0.918
Iteration 900
Current Accuracy: 0.9195833333333333
Iteration 950
Current Accuracy: 0.92095


## Kaggle Mnist Data

In [183]:
import struct
from array import array
from os.path import join

%matplotlib inline
import random
import matplotlib.pyplot as plt

#
# Set file paths based on added MNIST Datasets
#
input_path = '/home/alexanderscherrmann/Coding/python/ml/scratch_projects/deep_learning_scratch/'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1    
    for x in zip(images, title_texts):        
        image = x[0]        
        title_text = x[1]
        plt.subplot(rows, cols, index)        
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);        
        index += 1

class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (np.array(x_train).reshape(np.array(x_train).shape[0],-1)/255, np.array(y_train)),(np.array(x_test).reshape(np.array(x_test).shape[0],-1)/255, np.array(y_test))     

In [184]:
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(X_train, y_train), (X_val, y_val) = mnist_dataloader.load_data()

In [186]:
W1, b1, W2, b2 = grad_descent(X_train.T, y_train, 1000, 0.1, 10)

Iteration 0
Current Accuracy: 0.08956666666666667
Iteration 50
Current Accuracy: 0.14415
Iteration 100
Current Accuracy: 0.18565
Iteration 150
Current Accuracy: 0.22513333333333332
Iteration 200
Current Accuracy: 0.2742
Iteration 250
Current Accuracy: 0.29991666666666666
Iteration 300
Current Accuracy: 0.3221833333333333
Iteration 350
Current Accuracy: 0.34313333333333335
Iteration 400
Current Accuracy: 0.36336666666666667
Iteration 450
Current Accuracy: 0.38311666666666666
Iteration 500
Current Accuracy: 0.42766666666666664
Iteration 550
Current Accuracy: 0.4428
Iteration 600
Current Accuracy: 0.4577
Iteration 650
Current Accuracy: 0.4714333333333333
Iteration 700
Current Accuracy: 0.4842
Iteration 750
Current Accuracy: 0.49793333333333334
Iteration 800
Current Accuracy: 0.5105333333333333
Iteration 850
Current Accuracy: 0.5220333333333333
Iteration 900
Current Accuracy: 0.53415
Iteration 950
Current Accuracy: 0.5456666666666666


In [154]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Initialize with He initialization for better gradient flow
        self.params = {
            'W1': np.random.randn(hidden_size, input_size) * np.sqrt(2. / input_size),
            'b1': np.zeros((hidden_size, 1)),
            'W2': np.random.randn(output_size, hidden_size) * np.sqrt(2. / hidden_size),
            'b2': np.zeros((output_size, 1))
        }
        self.learning_rate = learning_rate
        
    def relu(self, Z):
        """ReLU activation function"""
        return np.maximum(0, Z)
    
    def relu_derivative(self, Z):
        """Derivative of ReLU function"""
        return Z > 0
    
    def softmax(self, Z):
        """Softmax activation function with numerical stability"""
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    
    def one_hot_encode(self, y, num_classes=None):
        """Convert label vector to one-hot encoded matrix"""
        if num_classes is None:
            num_classes = np.max(y) + 1
        one_hot = np.zeros((y.size, num_classes))
        one_hot[np.arange(y.size), y] = 1
        return one_hot.T
    
    def forward(self, X):
        """Forward pass through the network"""
        # Store intermediate values for backpropagation
        self.cache = {}
        
        # First layer
        self.cache['X'] = X
        self.cache['Z1'] = np.dot(self.params['W1'], X) + self.params['b1']
        self.cache['A1'] = self.relu(self.cache['Z1'])
        
        # Output layer
        self.cache['Z2'] = np.dot(self.params['W2'], self.cache['A1']) + self.params['b2']
        self.cache['A2'] = self.softmax(self.cache['Z2'])
        
        return self.cache['A2']
    
    def backward(self, y, output_activation=None):
        """Backward pass to compute gradients"""
        if output_activation is None:
            output_activation = self.cache['A2']
            
        m = y.shape[0]  # Number of examples
        
        # Convert labels to one-hot encoding if needed
        if len(y.shape) == 1:
            y_one_hot = self.one_hot_encode(y)
        else:
            y_one_hot = y.T  # Assuming y is already one-hot encoded but needs transposition
            
        # Output layer gradients
        dZ2 = output_activation - y_one_hot
        dW2 = (1 / m) * np.dot(dZ2, self.cache['A1'].T)
        db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
        
        # Hidden layer gradients
        dZ1 = np.dot(self.params['W2'].T, dZ2) * self.relu_derivative(self.cache['Z1'])
        dW1 = (1 / m) * np.dot(dZ1, self.cache['X'].T)
        db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
        
        # Store gradients
        self.gradients = {
            'W1': dW1, 'b1': db1,
            'W2': dW2, 'b2': db2
        }
        
        return self.gradients
    
    def update_parameters(self, gradients=None):
        """Update parameters using gradient descent"""
        if gradients is None:
            gradients = self.gradients
            
        for key in self.params:
            self.params[key] -= self.learning_rate * gradients[key]
    
    def predict(self, X):
        """Make predictions using the trained model"""
        output = self.forward(X)
        return np.argmax(output, axis=0)
    
    def calculate_accuracy(self, X, y):
        """Calculate accuracy of predictions"""
        predictions = self.predict(X)
        return np.mean(predictions == y)
    
    def calculate_loss(self, X, y):
        """Calculate cross-entropy loss"""
        m = y.shape[0]
        output = self.forward(X)
        
        # Convert y to one-hot if needed
        if len(y.shape) == 1:
            y_one_hot = self.one_hot_encode(y)
        else:
            y_one_hot = y.T
        
        # Avoid log(0) by adding a small epsilon
        epsilon = 1e-15
        log_probs = -np.log(output + epsilon) * y_one_hot
        loss = np.sum(log_probs) / m
        
        return loss
    
    def train(self, X, y, epochs=1000, print_every=100, X_val=None, y_val=None):
        """Train the network"""
        # Ensure X is in the correct shape (features, samples)
        if X.shape[0] < X.shape[1] and X.shape[0] == y.size:
            X = X.T  # Transpose if samples are in rows instead of columns
            
        history = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}
        
        for i in range(epochs):
            # Forward and backward passes
            self.forward(X)
            self.backward(y)
            self.update_parameters()
            
            # Record metrics
            if i % print_every == 0 or i == epochs - 1:
                train_acc = self.calculate_accuracy(X, y)
                train_loss = self.calculate_loss(X, y)
                history['train_acc'].append(train_acc)
                history['train_loss'].append(train_loss)
                
                print(f"Epoch {i}, Train Accuracy: {train_acc:.4f}, Loss: {train_loss:.4f}")
                
                # Validation if provided
                if X_val is not None and y_val is not None:
                    val_acc = self.calculate_accuracy(X_val, y_val)
                    val_loss = self.calculate_loss(X_val, y_val)
                    history['val_acc'].append(val_acc)
                    history['val_loss'].append(val_loss)
                    print(f"Validation Accuracy: {val_acc:.4f}, Loss: {val_loss:.4f}")
        
        return history


# Example usage:
def prepare_data(X, y):
    """Prepare data for the neural network"""
    # Convert to numpy arrays if they aren't already
    X = np.array(X)
    y = np.array(y)
    
    # Ensure X is in shape (features, samples)
    if X.shape[0] == y.size:
        X = X.T
    
    # Normalize features
    X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)
    
    return X, y

def train_test_split(X, y, test_size=0.2, random_state=None):
    """Split data into training and test sets"""
    if random_state is not None:
        np.random.seed(random_state)
    
    m = y.size
    indices = np.random.permutation(m)
    test_size = int(m * test_size)
    
    test_idx = indices[:test_size]
    train_idx = indices[test_size:]
    
    X_train = X[:, train_idx]
    y_train = y[train_idx]
    X_test = X[:, test_idx]
    y_test = y[test_idx]
    
    return X_train, X_test, y_train, y_test

In [161]:
np.array(X_tra).reshape(np.array(X_tra).shape[0],-1).shape

(60000, 784)

In [162]:
# Example:
X_tr = np.array(X_tra).reshape(np.array(X_tra).shape[0],-1)
X_data, y_data = prepare_data(X_tr, y_tra)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
model = NeuralNetwork(input_size=X_train.shape[0], hidden_size=64, output_size=len(np.unique(y_train)))
history = model.train(X_train, y_train, epochs=500, print_every=50, X_val=X_test, y_val=y_test)

/tmp/ipykernel_12529/3969906844.py:162: RuntimeWarning: invalid value encountered in divide
  X = (X - np.mean(X, axis=1, keepdims=True)) / np.std(X, axis=1, keepdims=True)


Epoch 0, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 50, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 100, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 150, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 200, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 250, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 300, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 350, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 400, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 450, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
Epoch 499, Train Accuracy: 0.0989, Loss: nan
Validation Accuracy: 0.0979, Loss: nan
